# An approach to fault tolerance

We'll begin by outlining a basic approach to fault-tolerant quantum computing based on quantum circuits and error correcting codes.

For the sake of this discussion, let us consider the following example of a quantum circuit.
This happens to be a teleportation circuit, including the preparation of the e-bit, but the specific functionality of the circuit is immaterial — it's just an example, and in actuality we're likely to be interested in significantly larger circuits.

![A teleportation circuit](/learning/images/courses/foundations-of-quantum-error-correction/fault-tolerant-quantum-computing/teleportation-circuit.svg)

A circuit like this one represents an ideal, and an actual implementation of it won't be perfect.
So what could go wrong?

The fact of the matter is that quite a lot could go wrong!
In particular, the state initializations, unitary operations, and measurements will all be imperfect;
and the qubits themselves will be susceptible to noise, including decoherence, at every point in the computation, even when no operations are being performed on them and they're simply storing quantum information.
That is to say, just about everything could go wrong.

There is one exception, though: Any *classical* computations that are involved are assumed to be perfect — because, practically speaking, classical computations are perfect.
For example, if we decide to use a surface code for error correction, and a classical perfect matching algorithm is run to compute corrections, we really don't need to concern ourselves with the possibility that errors in this classical computation will lead to a faulty solution.
As another example, quantum computations often necessitate classical pre- and post-processing, and these classical computations can safely be assumed to be perfect as well.

## Noise models

To analyze fault-tolerant implementations of quantum circuits, we require a precise mathematical model — a *noise model* — through which *probabilities* for various things to go wrong can be associated.
Hypothetically speaking, one could attempt to come up with a highly detailed, complicated noise model that aims to reflect the reality of what happens in a particular device.
But, if the noise model is too complicated or difficult to reason about, it will likely be of limited use.
For this reason, simpler noise models are much more typically considered.

One example of a simple noise model is the *independent stochastic noise model*, where errors or faults affecting different components at different moments in time — or, in other words, different *locations* in a quantum circuit — are assumed to be independent.
For instance, each gate might fail with a certain probability, an error might strike each stored qubit per unit time
with a different probability, and so on, with *no correlations* among the different possible errors.

Now, it is certainly reasonable to object to such a model, because there probably will be correlations among errors in real physical devices.
For instance, there might be a small chance of a catastrophic error that wipes out all the qubits at once.
Perhaps more likely, there could be errors that are localized but that nevertheless affect multiple components in a quantum computer.
Nobody suggests otherwise!
Nevertheless, the independent stochastic noise model does provide a simple baseline that captures the idea that nature is unpredictable but not malicious, and it isn't intentionally trying to ruin quantum computations.

Other, less forgiving noise models are also commonly studied.
For example, a common relaxation of the assumption of independence among errors affecting different locations in a quantum circuit is that *just the locations* of the errors are independent, but the actual errors affecting these locations could be correlated.

Regardless of what noise model is chosen, it should be recognized that *learning* about the errors that affect specific devices, and formulating new error models if the old ones lead us astray, could potentially be an important part of the development of fault-tolerant quantum computation.

## Fault-tolerant circuit implementations

Next we'll consider a basic strategy for fault-tolerant implementations of quantum circuits.
We'll use the teleportation circuit above as a running example to illustrate the strategy, though it could be applied to any quantum circuit.

Here's a diagram of a fault-tolerant implementation of our teleportation circuit.

![Fault-tolerant implementation of a teleportation circuit](/learning/images/courses/foundations-of-quantum-error-correction/fault-tolerant-quantum-computing/FT-teleportation-circuit.svg)

The individual components in this diagram and their connection to the original circuit are as follows.

1. State preparations, unitary gates, and measurements are not performed directly, as single operations, but rather are performed by so-called *gadgets*, which could each involve multiple qubits and multiple operations. In the diagram, gadgets are indicated by purple boxes labelled by whatever state preparation, gate, or measurement is to be implemented.

2. The *logical* qubits on which the original, ideal circuit is run are protected using a quantum error correcting code. Rather than acting directly on these logical qubits, the gadgets act on the *physical* qubits that encode them. The diagram suggests that five physical qubits are used for each logical qubit, as if the $5$-qubit code were being used, but the number could naturally be different. It is worth stressing that these logical qubits are *never* exposed; they spend their entire existence being protected by whatever quantum error correcting code we've chosen.

3. Error correction is performed repeatedly, as suggested by the blue boxes labeled "EC" in the diagram, throughout the computation. It is critically important that this is done both frequently and in parallel. As errors take place, <DefinitionTooltip definition="Entropy is a measure of randomness, uncertainty, or disorder in a system.">entropy</DefinitionTooltip> builds up, and constant work is required to remove it from the system at a high enough rate to allow the computation to function correctly.

There are therefore specific choices that must be made, including the selection of the gadgets as well as the quantum error correcting code itself.
Once these choices have been made, and assuming a particular noise model has been adopted, there is a fundamental question that we may ask ourselves:
Is this actually helping?
That is, are we making things better, or might we actually be making things worse?

If the rate of noise is too high, the entire process just suggested could very well be making things worse, just like the 9-qubit Shor code makes things worse for independent errors if the error probability on each qubit is above the break-even point.
If, however, the rate of noise is below a certain threshold, then all of this extra work will get us somewhere — and as we'll discuss toward the end of the lesson, paths open up for further error reduction.